In [2]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier 
import seaborn as sns
from sklearn.metrics import accuracy_score
import pickle
import datetime
import dateutil.relativedelta
from datetime import date
from dateutil import relativedelta

In [3]:
data = pd.read_csv(r'C:\Users\akobe\lighthouse-data-notes\Final-Data\final_data\all_merged_data_full_dob.csv', index_col = [0]) #index col makes sures another index column is not needed

In [4]:
data.head(2)

,nhl_id,elite_id,name,dob,draft_year,draft_season,prospect_gp,prospect_g,prospect_a,prospect_pts,...,position,prospect_category,amateur_league,amateur_team,nhl_games_played,200+games,birth_year,birth_month,birth_day,height_cm
0,8475166,9223,John Tavares,1990-09-20,2009,2008-2009,56,58,46,104,...,C,North American Skater,OHL,London,1017,1,1990,9,20,185.42
1,8475167,6007,Victor Hedman,1990-12-18,2009,2008-2009,45,7,16,23,...,D,European Skater,SWEDEN,Modo,964,1,1990,12,18,198.12


In [5]:
#drop unwanted columns 
data = data.drop(['nhl_id', 'elite_id', 'name', 'draft_season',  'nhl_games_played', 'pick_no', 'team', 'height'], axis=1)

In [6]:
data.head(2)

,dob,draft_year,prospect_gp,prospect_g,prospect_a,prospect_pts,prospect_pim,prospect_pm,birth_country,weight,shoots,position,prospect_category,amateur_league,amateur_team,200+games,birth_year,birth_month,birth_day,height_cm
0,1990-09-20,2009,56,58,46,104,54,10,CAN,209,L,C,North American Skater,OHL,London,1,1990,9,20,185.42
1,1990-12-18,2009,45,7,16,23,62,21,SWE,220,L,D,European Skater,SWEDEN,Modo,1,1990,12,18,198.12


In [7]:
data['age'] = (data['draft_year']-1) - data['birth_year']

In [8]:
def calc_prorated(df):
    ''' calculates player goals/assists/points to a 70 game season 
        (pts/games played) * 70'''
    prorated_games = 70
    
    df['pro_g'] = round((df['prospect_g']/df['prospect_gp']) * prorated_games, 2)
    df['pro_a'] = round((df['prospect_a']/df['prospect_gp']) * prorated_games, 2)
    df['pro_pts'] = round((df['prospect_pts']/df['prospect_gp']) * prorated_games, 2)
    

In [9]:
calc_prorated(data)

In [10]:
#define and apply league translation factor 
def apply_league_quality_translation(df):
    
    ''' adjusts players g/a/pts based on league quality
            multipy g/a/pts by the translation factor'''
    
    #translation factors 2016-2017
    KHL_transl = 0.77
    SHL_transl = 0.62 #Swedish hockey league 
    Finland_transl = 0.46 # Finland SM-liiga
    WCHA_transl = 0.44 # pre-2013 - Western Collegiate Hockey Association 
    NCHC_transl = 0.43 # National Collegiate Hockey Association 
    NLA_transl = 0.43 #Switzerland 
    hockey_east_transl = 0.38
    big_10_transl = 0.33
    CCHA_transl = 0.32 #Central Collegiate Hockey Assocaitiaion, now defunct
    OHL_transl = 0.31
    WHL_transl = 0.28
    QMJHL_transl = 0.25
    ECAC_transl = 0.23
    Czech_transl = 0.46
    other_transl = round((KHL_transl + SHL_transl +  Finland_transl + WCHA_transl + NCHC_transl + NLA_transl + hockey_east_transl + big_10_transl + 
                      CCHA_transl + OHL_transl + WHL_transl + QMJHL_transl + ECAC_transl  + Czech_transl)/14, 2)

    
    #applying the correct translation factor 
    translation_factor = []
    
    for i in df['amateur_league']:
        if i == 'OHL':
            translation_factor.append(OHL_transl)
        elif i == 'SWEDEN':
            translation_factor.append(SHL_transl)
        elif i == 'SWEDEN-2':
            translation_factor.append(SHL_transl)
        elif i == 'WHL':
            translation_factor.append(WHL_transl)
        elif i == 'QMJHL':
            translation_factor.append(QMJHL_transl)
        elif i == 'WCHA':
            translation_factor.append(WCHA_transl)
        elif i == 'FINLAND':
            translation_factor.append(Finland_transl)
        elif i == 'CCHA':
            translation_factor.append(CCHA_transl)
        elif i == 'H-EAST':
            translation_factor.append(hockey_east_transl)
        elif i == 'KHL':
            translation_factor.append(KHL_transl)
        elif i == 'BIG10':
            translation_factor.append(big_10_transl)
        elif i == 'CZECH':
            translation_factor.append(Czech_transl)
        elif i == 'NCHC':
            translation_factor.append(NCHC_transl)
        else: 
            translation_factor.append(other_transl)
     
    #creates translation factor column
    df['translation_factor'] = translation_factor
    
    #calculates g/a/pts based on league translation factor 
    df['transl_g'] = round(df['pro_g'] * df['translation_factor'], 2)
    df['transl_a'] = round(df['pro_a'] * df['translation_factor'], 2)
    df['transl_pts'] = round(df['pro_pts'] * df['translation_factor'], 2)
    

In [11]:
#apply the function
apply_league_quality_translation(data)

In [38]:
def calc_draft_date(df):
    '''gives a column of draft date cut off '''
    
    #draft dates 
    draft_date = []
    
    for i in df['draft_year']: 
        if i == 2009:
            draft_date.append(datetime.date(2009, 9, 15))
        elif i == 2010:
            draft_date.append(datetime.date(2010, 9, 15))
        elif i == 2011:
            draft_date.append(datetime.date(2011, 9, 15))
        elif i == 2012:
            draft_date.append(datetime.date(2012, 9, 15))
        elif i == 2013: 
            draft_date.append(datetime.date(2013, 9, 15))
        elif i == 2014:
            draft_date.append(datetime.date(2014, 9, 15))
        elif i == 2015:
            draft_date.append(datetime.date(2015, 9, 15))
        elif i == 2016:
            draft_date.append(datetime.date(2016, 9, 15))
        elif i == 2017:
            draft_date.append(datetime.date(2017, 9, 15))
        elif i == 2018:
            draft_date.append(datetime.date(2018, 9, 15))

    #creates translation factor column
    df['draft_date'] = draft_date

In [39]:
calc_draft_date(data)

In [49]:
def calc_draft_age(df):        
    #converts columns to datetime 
    df['dob'] = pd.to_datetime(df['dob'])
    df['draft_date'] = pd.to_datetime(df['draft_date'])

    delta = []
    draft_age_years = []
    draft_age_months = []
    
    #get the relative time in between dates 
    for index, row in df.iterrows():
        delta.append(relativedelta.relativedelta(row['draft_date'], row['dob']))
    
    df['delta'] = delta
    
    for index, i in df.iterrows():
        #subtracts 1 year from age as year is considered as the year before the draft (when player is 17)
        draft_age_years.append((i['delta'].years) - 1)
        draft_age_months.append(i['delta'].months)
    
    df['draft_age_years'] = draft_age_years
    df['draft_age_months'] = draft_age_months
    
    '''calculates the age in years and months at the draft cut-off date (Sept 15 of the year of the 
    draft)
      
    dob and draft date converted to datetime, 
    time in between dates calculated '''

In [15]:
calc_draft_age(data)

In [50]:
def calc_age_code(df): 
    '''combines age in year and months into a string to create an age code 
    which will later determine the age mupltiplier to apply'''
    
    df['age_code'] = df['draft_age_years'].astype(str) +""+ df['draft_age_months'].astype(str)

In [17]:
calc_age_code(data)

In [51]:
def apply_age_multiplier(df): 
    '''assigns an age multiplier''' 
    age_multiplier = []
    
    #assigns multiplier based on age code 
    for i in df['age_code']: 
        if i == '170':
            age_multiplier.append(1.00)
        elif i == '171':
            age_multiplier.append(0.995)
        elif i == '172':
            age_multiplier.append(0.990)
        elif i == '173':
            age_multiplier.append(0.986)
        elif i == '174':
            age_multiplier.append(0.981)
        elif i == '175':
            age_multiplier.append(0.976)
        elif i == '176':
            age_multiplier.append(0.971)
        elif i == '177':
            age_multiplier.append(0.967)
        elif i == '178':
            age_multiplier.append(0.962)
        elif i == '179':
            age_multiplier.append(0.958)
        elif i == '1710':
            age_multiplier.append(0.953)
        elif i == '1711':
            age_multiplier.append(0.949)
        elif i == '180':
            age_multiplier.append(0.840)
        elif i == '181':
            age_multiplier.append(0.832)
        elif i == '182':
            age_multiplier.append(0.824)
        elif i == '183':
            age_multiplier.append(0.817)
        elif i == '184':
            age_multiplier.append(0.809)
        elif i == '185':
            age_multiplier.append(0.802)
        elif i == '186':
            age_multiplier.append(0.795)
        elif i == '187':
            age_multiplier.append(0.788)
        elif i == '188':
            age_multiplier.append(0.781)
        elif i == '189':
            age_multiplier.append(0.774)
        elif i == '1810':
            age_multiplier.append(0.767)
        else:
            age_multiplier.append(0.760)   
    
    #creates a dataframe column with the age multiplier 
    df['age_multiplier'] = age_multiplier

In [19]:
apply_age_multiplier(data)

In [20]:
def calc_eq_points(df):
    ''' applies the  age multiplier to g/a/pts '''
    
    #calculates g/a/pts based on league translation factor 
    df['eq_g'] = round(df['transl_g'] * df['age_multiplier'], 2)
    df['eq_a'] = round(df['transl_a'] * df['age_multiplier'], 2)
    df['eq_pts'] = round(df['transl_pts'] * df['age_multiplier'], 2)

In [21]:
calc_eq_points(data)

In [22]:
#drop unwanted columns 
data = data.drop(['draft_date', 'delta', 'draft_age_years',  'draft_age_months', 'age_code','prospect_g', 'prospect_a',
                  'prospect_pts', 'pro_g', 'pro_a', 'pro_pts', 'transl_g', 'transl_a', 'transl_pts', 'age_multiplier', 'birth_year', 
                     'birth_day', 'age'], axis=1)

In [24]:
#calculate OGVT

def calc_offensive_GVT(df):
    ''' calculates a modified offensive GVT (goals vs threshold)
    subtract eq_pts by baseline (15 forwards, 5 defencemen), 
    divde result by 6-forwards, 7-defensmen,
    subtract another threshold value (3.5 forwards, 1.75 defensemen)'''
    
    oGVT = []
    
    for index, row in df.iterrows():
        if row.position == 'D':
            oGVT.append(round(((row.eq_pts-5)/7) - 1.75, 2))      
        else:
            oGVT.append(round(((row.eq_pts-15)/6) - 3.5, 2)) 
            
    data['oGVT'] = oGVT

In [25]:
calc_offensive_GVT(data)

In [26]:
def calc_defensive_GVT(df):
    ''' calculates a modified defensive GVT (goals vs threshold)
    start with baseline number (3.5 for defensemen, 1.0 for forwards)
    apply league quality multiplier 
    subtract a threshold value (2.00 for defensemen, 0.25 for forwards)'''
    
    def_baseline = 3.5
    off_baseline = 1.0
    
    dGVT = []
    
    for index, row in df.iterrows():
        if row.position == 'D':
            dGVT.append(round((row.translation_factor*def_baseline)-2, 2))
        else: 
            dGVT.append(round((row.translation_factor*off_baseline)-0.25, 2))
             
    data['dGVT'] = dGVT

In [27]:
calc_defensive_GVT(data)

In [53]:
def calc_overall_GVT(df):
    '''calculates overall GVT by adding oGVT to dGVT '''
    
    df['overall_GVT'] = df['oGVT'] + df['dGVT']

In [29]:
calc_overall_GVT(data)

In [35]:
def positions(df):
    ''' provides position as F or D - converts C, LW, RW to F'''
    
    position_2 = []
    
    for i in df['position']: 
        if i == 'D':
            position_2.append('D')
        else:
            position_2.append('F')
    
    df['position_2'] = position_2

In [31]:
positions(data)

In [33]:
data = data.drop(['position'], axis=1)

In [36]:
data.head(2)

,dob,draft_year,prospect_gp,prospect_pim,prospect_pm,birth_country,weight,shoots,prospect_category,amateur_league,...,birth_month,height_cm,translation_factor,eq_g,eq_a,eq_pts,oGVT,dGVT,overall_GVT,position_2
0,1990-09-20,2009,56,54,10,CAN,209,L,North American Skater,OHL,...,9,185.42,0.31,21.33,16.91,38.24,0.37,0.06,0.43,F
1,1990-12-18,2009,45,62,21,SWE,220,L,European Skater,SWEDEN,...,12,198.12,0.62,6.49,14.84,21.34,0.58,0.17,0.75,D


In [37]:
#save the data to csv 
#data.to_csv('transformed_data.csv')

In [6]:
###### 

draft_date = datetime.date(2023, 9, 15)
birth_date = datetime.date(2005, 7, 17)


draft_age = relativedelta.relativedelta(draft_date, birth_date)


draft_age_years = draft_age.years - 1
draft_age_months = draft_age.months

print(f'draft year {draft_age_years}, draft months {draft_age_months}')

draft year 17, draft months 1
